In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [49]:
#Classifying cat vs dogs
# Features : Ear_shape, Face_shape, Whiskers
class Node:
    def __init__(self,value=None,threshold=None,left=None,right=None,feature_index=None):
        self.value=value
        self.left=left
        self.right=right
        self.threshold=threshold
        self.feature_index=feature_index

class decision_tree:
    
    def __init__(self,depth,min_charnum):
        self.X_train = np.array([[1, 1, 1],
                            [0, 0, 1],
                            [0, 1, 0],
                            [1, 0, 1],
                            [1, 1, 1],
                            [1, 1, 0],
                            [0, 0, 0],
                            [1, 1, 0],
                            [0, 1, 0],
                            [0, 1, 0]])

        self.y_train = np.array([1, 1, 0, 0, 1, 1, 0, 1, 0, 0])
        self.row=len(self.X_train[:,0])
        self.column=len(self.X_train[0])
        self.depth=depth
        self.tree=None
        self.min_charnum=min_charnum

    def entropy(self,p):
        if p==0 or p==1:
            return 0
        else:
            H = -p*np.log2(p) - (1-p)*np.log2(1-p)
        return H

    def split(self,X,feature_index):        # X - Indexes in the root node ,  y - y_train
        left_indices=[]
        right_indices=[]
        x=self.X_train[:,feature_index]       #Extracting the column of the feature
        for i in X:
            if x[i]==1:
                left_indices.append(i)
            elif x[i]==0:
                right_indices.append(i)
        return left_indices,right_indices

    def i_gain(self,left_indices,right_indices,X):
        if len(left_indices)==0 or len(right_indices)==0:
            return 0
        p_left=sum(self.y_train[left_indices])/len(left_indices)
        p_right=sum(self.y_train[right_indices])/len(right_indices)
        p_parent=sum(self.y_train[X])/len(X)
        w_left=len(left_indices)/len(X)
        w_right=len(right_indices)/len(X)
        return self.entropy(p_parent)-(w_left*self.entropy(p_left) + w_right*self.entropy(p_right))

    def best_feature(self,X):
        max_gain=-1
        index=0
        for i in range(self.column):
            left_indices,right_indices=self.split(X,i)
            gain=self.i_gain(left_indices,right_indices,X)
            if(gain>max_gain):
                index=i
                max_gain=gain
        return index

    def recursive(self,X,current_depth):
        #Put base condition: len(X)< a threshold and also the depth
        if len(X)<self.min_charnum or current_depth>=self.depth:
            return Node(value=self.most_common_label(X))
        sum=0         # Base case to check if all the elements in the node is of the same class 
        for i in X:
            sum+= self.y_train[i]
        if sum==0 or sum==len(X):
            return Node(value=self.most_common_label(X))
    
            
        index=self.best_feature(X)
        left_indices,right_indices=self.split(X,index)

        if not left_indices or not right_indices:
            return Node(value=self._most_common_label(X))
            
        left_node = self.recursive(left_indices,current_depth+1)
        right_node = self.recursive(right_indices,current_depth+1)

        return Node(left=left_node,right=right_node,feature_index=index)

    def most_common_label(self,X):
        count=np.bincount(self.y_train[X])
        return count.argmax()
        
     # To classify a new input (i.e., determine whether it is a dog or a cat)
     #you need to traverse the trained decision tree using the feature values of the input. 

    def predict(self, X):
        return [self._traverse_tree(x, self.tree) for x in X]

    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature_index] == 1:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

        

    

def main():
    object=decision_tree(15,2)
    X_node=np.arange(object.row)     # Initially all the indexes of X_train is considered 
    object.tree = object.recursive(X_node,0)

    # Example input to predict
    test_data = np.array([[1, 0, 0], [0, 1, 1], [1, 1, 0],[1,1,1]])
    predictions = object.predict(test_data)
    print("Predictions:", predictions)
main()



Predictions: [0, 1, 1, 1]
